# Running a CPL recipe from Python using PyEsoRex

In [1]:
from matplotlib import pyplot as plt
from astropy.visualization import PercentileInterval, ImageNormalize

plt.rcParams['figure.figsize'] = 12, 12

## Importing the Pyesorex class

In [2]:
from pyesorex.pyesorex import Pyesorex

ImportError: /home/ahorton/miniconda3/envs/cpl_py310/bin/../lib/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /home/ahorton/miniconda3/envs/cpl_py310/lib/python3.10/site-packages/cpl.cpython-310-x86_64-linux-gnu.so)

## Creating a Pyesorex instance

If done with no arguments it will look for try to load a `pyesorex.rc` config file in the default location, `~/.pyesorex`. If there is no config file at the default path the parameters will use the default values.

In [3]:
p = Pyesorex()

NameError: name 'Pyesorex' is not defined

## Inspecting the current PyEsoRex parameter values

In [ ]:
print(p.parameters)

## Creating another Pyesorex instance

An alternative config file can be specified when instantiating the Pyesorex object.

In [ ]:
p = Pyesorex(config='pyesorex.rc')

In [ ]:
print(p.parameters)

## Creating a new config file

By default `write_config()` creates a config file from the current PyEsoRex parameter values. If the file already exists it will be moved to `<filename>.bak` before the new file is created. The config path is an optional argument, if not given it will create the config file at the default location, `~/.pyesorex/pyesorex.rc`.

In [ ]:
p.write_config("demo.rc")

## Creating a new default config file

With `defaults=True` `write_config()` creates a config file using the built in default parameter values.

In [ ]:
p.write_config("demo_default.rc", defaults=True)

## Reading a config file

Config files can be read at any time.

In [ ]:
p.read_config("demo.rc")

In [ ]:
print(p.parameters)

## Listing available recipes

`get_recipes()` returns a dictionary of the names of all the C and Python recipes found in the recipe directories set in `parameters['recipe_dirs']` together with some of their metadata.

`get_recipes_text()` returns a formatted string listing the names of available recipes and their synopses.

In [ ]:
p.get_recipes()

In [ ]:
print(p.get_recipes_text())

## Loading a recipe

If done with no arguments it will try to load a `<recipe_name>.rc` config file in the default location, `~/.pyesorex`. If there is no config file at the default path the parameters will use the default values.

In [ ]:
p.load_recipe("gimasterdark")

In [ ]:
print(p.recipe_parameters)

Recipes can also be loaded by assigning to the `recipe` property, however in this case there is no option to specify a recipe config file at the same time.

In [ ]:
p.recipe = "gimasterbias"

In [ ]:
p.recipe

In [ ]:
print(p.recipe_parameters)

## Changing recipe settings

The `recipe_parameters` attribute has an `update()` method that takes a dictionary of `<parameter_name>: <new_value>` pairs.

In [ ]:
p.recipe_parameters.update({"giraffe.stacking.method":"median", "giraffe.masterbias.bpm.create":False})

In [ ]:
print(p.recipe_parameters)

## Creating a recipe config file

As with PyEsoRex config files this will create a config file using the current setting, unless `defaults` is set to `True`. Again the config path is an optional argument, if not given it will create the config file at the default location, `~/.pyesorex/<recipe_name>.rc`.

In [ ]:
p.write_recipe_config("gimasterbias.rc")

In [ ]:
p.write_recipe_config("gimasterbias_defaults.rc", defaults=True)

## Reading a recipe config file

In [ ]:
p.read_recipe_config("gimasterbias_defaults.rc")

In [ ]:
print(p.recipe_parameters)

## Setting the SOF location

The SOF file lists the paths and types of the FITS files that the recipe will process.

In [ ]:
p.sof_location = '0001.gimasterbias.sof'

In [ ]:
p.sof_location

## Running the recipe

In [ ]:
result = p.run()

## Inspecting the result

The `run()` method returns a `cpl.ui.FrameSet` object which contains the output `cpl.ui.Frame`s of the recipe. Each `Frame` contains the path to an output FITS file, and some basic metadata.

In [ ]:
result

In [ ]:
for frame in result:
    print(repr(frame))

### Converting the result to an HDU list

`Frame` objects can open the corresponding FITS file as an `astropy.io.fits.HDUList` object. 

In [ ]:
master_bias_hdulist = result[0].as_hdulist()

In [ ]:
master_bias_hdulist

In [ ]:
master_bias_hdulist[0].header

In [ ]:
master_bias_hdulist[0].data

### Converting the result to CCDData

`Frame` can also open the corresponding FITS file as an `astropy.nddata.CCDData` object.

In [ ]:
master_bias = result[0].as_ccddata(unit="adu")

In [ ]:
master_bias

In [ ]:
norm = ImageNormalize(master_bias, PercentileInterval(99.0))
plt.imshow(master_bias, norm=norm, interpolation='nearest')
plt.colorbar();